# Memory Collections

In [1]:
from pydantic import BaseModel, Field

class InteractionDirective(BaseModel):
    directive: str = Field(
        description=(
            "An independent, atomic instruction that describes one specific way the assistant should adapt its interaction to the user. "
            "Each directive should focus on a single behavior or style — do not combine multiple preferences into one. "
            "Directives must be phrased as imperatives. "
            "Examples: "
            "'Explain technical topics using a step-by-step format.', "
            "'Use concise code examples.', "
            "'Avoid adding explanations unless explicitly requested.', "
            "'Include inline comments in code when requested.', "
            "'Ask if the user wants a deeper explanation after giving an answer.'"
        )
    )

class InteractionStyle(BaseModel):
    directives: list[InteractionDirective] = Field(
        description="A collection of interaction-relevant insights to guide how the assistant should respond to this user."
    )

## Creating collection

In [2]:
from trustcall import create_extractor
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
# trustcall_extractor = create_extractor(model, tools=[UserFacts])

trustcall_extractor = create_extractor(
    model,
    tools=[InteractionStyle],
    tool_choice="InteractionStyle"
)


# Invoke the extractor
result = trustcall_extractor.invoke([HumanMessage(content="""
Hey, how are you? I am Evgeny, I am a junior software engineer and I need your help with 
my project. This is a TaskManager Java Spring Boot application. I do not get how
to configure security there!
Also, I often come back to these answers later, so please include comments inside any code you share.
""")])

In [3]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  InteractionStyle (call_nLL2pCgE9EzXLTGiNxYmHiAO)
 Call ID: call_nLL2pCgE9EzXLTGiNxYmHiAO
  Args:
    directives: [{'directive': 'Explain technical topics using a step-by-step format.'}, {'directive': 'Include inline comments in code when requested.'}, {'directive': 'Ask if the user wants a deeper explanation after giving an answer.'}]


In [4]:
result["responses"][0].directives

[InteractionDirective(directive='Explain technical topics using a step-by-step format.'),
 InteractionDirective(directive='Include inline comments in code when requested.'),
 InteractionDirective(directive='Ask if the user wants a deeper explanation after giving an answer.')]

### let's save initial memories into long-term memory

In [5]:
from langgraph.store.memory import InMemoryStore

# Initialize the in-memory store
in_memory_store = InMemoryStore()

# Namespace for the memory to save
user_id = "1"
namespace_for_directives = (user_id, "directives")

for i, directive in enumerate(result["responses"][0].directives):
    in_memory_store.put(namespace_for_directives, f"directive-{i}", directive.model_dump())

In [6]:
for directive in in_memory_store.search(namespace_for_directives):
    print(directive.dict())

{'namespace': ['1', 'directives'], 'key': 'directive-0', 'value': {'directive': 'Explain technical topics using a step-by-step format.'}, 'created_at': '2025-08-18T14:31:49.197195+00:00', 'updated_at': '2025-08-18T14:31:49.197195+00:00', 'score': None}
{'namespace': ['1', 'directives'], 'key': 'directive-1', 'value': {'directive': 'Include inline comments in code when requested.'}, 'created_at': '2025-08-18T14:31:49.197195+00:00', 'updated_at': '2025-08-18T14:31:49.197195+00:00', 'score': None}
{'namespace': ['1', 'directives'], 'key': 'directive-2', 'value': {'directive': 'Ask if the user wants a deeper explanation after giving an answer.'}, 'created_at': '2025-08-18T14:31:49.197195+00:00', 'updated_at': '2025-08-18T14:31:49.197195+00:00', 'score': None}


## Updating collection

The same as for user profile we want to be able to update the collection, not recreate it everytime. So what we need:

1. update already existing memory items if there is an update for the fact
2. create new memory items if there is a new fact

a way how to do this with Trustcall is to use `enable_inserts=True` and single elements Schema. Another thing we have to solve - to provide memory ids for update. 

In [7]:
from trustcall import create_extractor

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools=[InteractionDirective],
    tool_choice="InteractionDirective",
    enable_inserts=True
)

In [8]:
# prepare existing facts
schema = "InteractionDirective"
existing_directives = in_memory_store.search(namespace_for_directives)
memories = (
    [(existing_directive.key, schema, existing_directive.value) for existing_directive in existing_directives] 
    if existing_directives else None
)
memories

[('directive-0',
  'InteractionDirective',
  {'directive': 'Explain technical topics using a step-by-step format.'}),
 ('directive-1',
  'InteractionDirective',
  {'directive': 'Include inline comments in code when requested.'}),
 ('directive-2',
  'InteractionDirective',
  {'directive': 'Ask if the user wants a deeper explanation after giving an answer.'})]

In [9]:
from langchain_core.messages import HumanMessage, AIMessage

conversation = [
    HumanMessage(content="""
    Hey, how are you? I’m Evgeny, I’m a software engineer and I need your help with my project.
    This is a TaskManager Java Spring Boot application. I do not get how to configure security there!
    Also, I often come back to these answers later, so please include comments inside any code you share.
    """), 

    AIMessage(content="""
    Hey Evgeny! I’d be glad to help. Spring Security can be a bit tricky at first. Do you want a quick step-by-step setup example, 
    or would you prefer a deeper explanation of how it works?
    """), 

    HumanMessage(content="""
    Sorry if I sound a bit stressed — I’m not very experienced with Spring Security.
    If you could break things down step by step, that would really help.
    """),

    AIMessage(content="""
    Totally understood! No worries at all — I’ll walk you through it one step at a time.
    Let’s start with the basic configuration: you’ll need to add the Spring Security dependency first. Here's how...
    """), 

    HumanMessage(content="""
    Thanks, but could you skip the long explanation? Just tell me exactly what I need to do.
    """),
]

# Update the instruction
system_msg = """
Update existing directives about the user and create new ones based on the following conversation, 
ensuring each directive reflects one specific communication preference or behavior the assistant should follow.
"""


result = trustcall_extractor.invoke({"messages": [system_msg] + conversation, 
                                     "existing": memories})

In [10]:
# Messages from the model indicate two tool calls were made
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  InteractionDirective (call_aDIzktGWYyNI40XmnRVvACA9)
 Call ID: call_aDIzktGWYyNI40XmnRVvACA9
  Args:
    directive: Break down technical topics into clear steps without lengthy explanations.
  InteractionDirective (call_gz3W6H84yG8GoHV4sCikzzyq)
 Call ID: call_gz3W6H84yG8GoHV4sCikzzyq
  Args:
    directive: Ask if the user wants a deeper explanation or prefers concise instructions after giving an answer.
  InteractionDirective (call_mRu0TzqUFNp9RYwlPjOGxgs6)
 Call ID: call_mRu0TzqUFNp9RYwlPjOGxgs6
  Args:
    directive: Apologize for any stress expressed by the user to create a supportive atmosphere.


In [11]:
result["response_metadata"]

[{'id': 'call_aDIzktGWYyNI40XmnRVvACA9', 'json_doc_id': 'directive-0'},
 {'id': 'call_gz3W6H84yG8GoHV4sCikzzyq', 'json_doc_id': 'directive-2'},
 {'id': 'call_mRu0TzqUFNp9RYwlPjOGxgs6'}]

In [12]:
# Print the results
print("Updated and new person records:")
for r, rmeta in zip(result["responses"], result["response_metadata"]):
    print(f"ID: {rmeta.get('json_doc_id', 'New')}")
    print(r.model_dump_json(indent=2))
    print()

Updated and new person records:
ID: directive-0
{
  "directive": "Break down technical topics into clear steps without lengthy explanations."
}

ID: directive-2
{
  "directive": "Ask if the user wants a deeper explanation or prefers concise instructions after giving an answer."
}

ID: New
{
  "directive": "Apologize for any stress expressed by the user to create a supportive atmosphere."
}



## ChatBot with updateable directions from memory (in studio)

see [studio/directive_memory_bot.py](studio/directive_memory_bot.py)